# Compile RV measurements for all stars
I will use the MultiNest output to recreate the MCMC chains for primary star RV, and use the cross-correlation functions to measure the companion rv (where applicable)

In [6]:
import Fitters
import h5py
import glob
import os
import HelperFunctions
from astropy.io import fits
import pandas as pd

home = os.environ['HOME']

In [7]:
# Find all the base directories for each instrument
instrument_dirs = dict(TS23='{}/School/Research/McDonaldData'.format(home),
                       CHIRON='{}/School/Research/CHIRON_data'.format(home),
                       HRS='{}/School/Research/HET_data'.format(home),
                       IGRINS='{}/School/Research/IGRINS_data'.format(home))
basedirs = {inst: [n[:-4] for n in glob.glob('{}/RVFitter_flattened/*-.txt'.format(instrument_dirs[inst]))] for inst in instrument_dirs.keys()}

# Collect the model files
model_files = dict(TS23='/Volumes/DATADRIVE/Kurucz_Grid/TS23_grid_air.hdf5',
                   HRS='/Volumes/DATADRIVE/Kurucz_Grid/HRS_grid_full.hdf5',
                   CHIRON='/Volumes/DATADRIVE/Kurucz_Grid/CHIRON_grid_air.hdf5',
                   IGRINS='/Volumes/DATADRIVE/Kurucz_Grid/IGRINS_grid_air.hdf5')

# Get the flatten-fit information
df_list = []
for instrument in instrument_dirs.keys():
    fname = '{}/Flatten.log'.format(instrument_dirs[instrument])
    df = pd.read_csv(fname, header=None, names=['fname', 'star', 'date', 'teff', 'logg', 'rv'])
    df.fname = df.fname.map(lambda s: '{}/{}'.format(instrument_dirs[instrument], s))
    df.date = df.date.map(lambda s: s.replace('-', ''))
    df['Instrument'] = instrument
    df_list.append(df.copy())

flatten_df = pd.concat(df_list)

In [ ]:
# Get the MCMC chains, and store them in an HDF5 file.

with h5py.File('RV_data.h5', 'w') as outfile:
    for instrument in basedirs.keys():
    #for instrument in ['CHIRON']:
        inst_grp = outfile.require_group(instrument)
        for basename in basedirs[instrument]:
            print(instrument, basename)
            # Get the star name and observation date from the filename
            starname, date = os.path.basename(basename).split('_')
            date = date[:-1]
            if starname.startswith('HIP'):
                starname = 'HIP {}'.format(starname[3:])
            elif starname.startswith('HR'):
                starname = 'HR {}'.format(starname[2:])
            elif starname.startswith('ADS'):
                starname = 'ADS {}'.format(starname[3:])
            else:
                print("Weird starname: {}".format(starname))
                continue
            print(starname, date)
            
            # Get the path to the original fits file 
            star_data = flatten_df.loc[(flatten_df.star == starname) & 
                                       (flatten_df.date == date) & 
                                       (flatten_df.Instrument == instrument)]
            filename = star_data.fname.item()
            teff = star_data.teff.item()
            logg = star_data.logg.item()
            feh = 0.0
            orders = HelperFunctions.ReadExtensionFits(filename)

            # Set up the fitter
            fitter = Fitters.RVFitter(orders, model_library=model_files[instrument],
                                      T=teff, logg=logg, feh=feh)

            # Fit (these are already fit, so this just gets the MCMC chains)
            fitter.fit(backend='multinest', n_live_points=1000, basename=basename, overwrite=False, init_MPI=False)
            
            # Output to the HDF5 file
            star_grp = inst_grp.require_group(starname)
            dataset = star_grp.create_dataset(date, data=fitter.samples.values, maxshape=(None, 4))
            
            # Set attributes to summary values (so I don't need to read in the data if I only care about mean values)
            dataset.attrs['column_names'] = list(fitter.samples.columns)
            dataset.attrs['Teff'] = teff
            dataset.attrs['logg'] = logg
            dataset.attrs['rv'] = fitter.samples.RV.mean()
            dataset.attrs['rv_err'] = fitter.samples.RV.std()
            dataset.attrs['vsini'] = fitter.samples.vsini.mean()
            dataset.attrs['vsini_err'] = fitter.samples.vsini.std()
            dataset.attrs['epsilon'] = fitter.samples.epsilon.mean()
            dataset.attrs['epsilon_err'] = fitter.samples.epsilon.std()
            dataset.attrs['path'] = filename.replace(home, '~')
            

('CHIRON', '/Users/kgulliks/School/Research/CHIRON_data/RVFitter_flattened/HIP100881_20130807-')
('HIP 100881', '20130807')
Determine Chunk Log: Wl is 262144
('CHIRON', '/Users/kgulliks/School/Research/CHIRON_data/RVFitter_flattened/HIP101589_20130605-')
('HIP 101589', '20130605')
Determine Chunk Log: Wl is 262144
('CHIRON', '/Users/kgulliks/School/Research/CHIRON_data/RVFitter_flattened/HIP101867_20130716-')
('HIP 101867', '20130716')
Determine Chunk Log: Wl is 262144
('CHIRON', '/Users/kgulliks/School/Research/CHIRON_data/RVFitter_flattened/HIP10320_20130828-')
('HIP 10320', '20130828')
Determine Chunk Log: Wl is 262144
('CHIRON', '/Users/kgulliks/School/Research/CHIRON_data/RVFitter_flattened/HIP104019_20130701-')
('HIP 104019', '20130701')
Determine Chunk Log: Wl is 262144
('CHIRON', '/Users/kgulliks/School/Research/CHIRON_data/RVFitter_flattened/HIP104139_20130605-')
('HIP 104139', '20130605')
Determine Chunk Log: Wl is 262144
('CHIRON', '/Users/kgulliks/School/Research/CHIRON_dat